In [1]:
import stable_baselines3
import gymnasium as gym
import numpy as np

from stable_baselines3 import DQN
from stable_baselines3.common.env_util import make_atari_env
from stable_baselines3.common.vec_env import VecFrameStack
#from stable_baselines.common.atari_wrappers import make_atari
from stable_baselines3.dqn import MlpPolicy, CnnPolicy

#atari:
#   env_wrapper:
#     - stable_baselines3.common.atari_wrappers.AtariWrapper
#   frame_stack: 4
#   policy: 'CnnPolicy'
#   n_timesteps: !!float 1e7
#   buffer_size: 100000
#   learning_rate: !!float 1e-4
#   batch_size: 32
#   learning_starts: 100000
#   target_update_interval: 1000
#   train_freq: 4
#   gradient_steps: 1
#   exploration_fraction: 0.1
#   exploration_final_eps: 0.01
#   # If True, you need to deactivate handle_timeout_termination
#   # in the replay_buffer_kwargs
#   optimize_memory_usage: False


# There already exists an environment generator that will make and wrap atari environments correctly.
env = make_atari_env("BreakoutNoFrameskip-v4", n_envs=4, seed=0, env_kwargs={"render_mode" : "rgb_array"})
obs = env.reset()
print(obs.shape)
# Stack 4 frames
env = VecFrameStack(env, n_stack=4)
obs = env.reset()
print(obs.shape)

model = DQN("CnnPolicy", env, buffer_size=1000, verbose=1)#cnn neural network for images #verbos for outputs

model.learn(total_timesteps=10_000)



(4, 84, 84, 1)
(4, 84, 84, 4)
Using cpu device
Wrapping the env in a VecTransposeImage.
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.848    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 477      |
|    time_elapsed     | 0        |
|    total_timesteps  | 160      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.681    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 539      |
|    time_elapsed     | 0        |
|    total_timesteps  | 336      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.479    |
| time/               |          |
|    episodes         | 12       |
|    fps              | 487      |
|    time_elapsed     | 1        |
|    total_timesteps  | 548      |
--